In this notebook we give a proof of concept of unitary compiling using TFQ. 

In [2]:
%load_ext autoreload
%autoreload 2

import sympy 
import numpy as np 
import pandas as pd 
import tensorflow as tf
from utilities.circuit_database import CirqTranslater
from utilities.templates import *
from utilities.variational import Minimizer
import matplotlib.pyplot as plt 
import tensorflow_quantum as tfq
import cirq
from utilities.compiling import *
from utilities.misc import *
from utilities.simplifier import Simplifier
from utilities.discrimination import *
from utilities.idinserter import IdInserter
from utilities.evaluator import Evaluator
from utilities.gate_killer import GateKiller
from ast import literal_eval


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
translator = CirqTranslater(4, untouchable_blocks = [1,0])
inserter = IdInserter(translator.n_qubits, untouchable_blocks=translator.untouchable_blocks)

simplifier = Simplifier(translator)
etas = [0.01, 1.]
minimizer = Minimizer(translator, mode="vqe", hamiltonian="TFIM", params=[1.0, 0.])
killer = GateKiller(translator, mode="vqe", hamiltonian="TFIM", params=[1.0, 0.])

computing ground state energy...


In [15]:
circuit_db = concatenate_dbs([u2_layer(translator, block_id=k) for k in range(3)])
circuit, circuit_db = translator.give_circuit(circuit_db)

In [16]:
import tensorflow_quantum as tfq

In [18]:
cc = tfq.convert_to_tensor([circuit])

In [25]:
def prepare_circuit_vqe(circuit_db):
    trainable_symbols = translator.get_trainable_symbols(circuit_db)
    trainable_param_values = translator.get_trainable_params_value(circuit_db)
    return trainable_symbols, trainable_param_values

In [30]:
cost, resolver, training_history = minimizer.minimize([circuit], symbols = trainable_symbols, parameter_values = trainable_param_values )

In [37]:
minimized_circuit_db = translator.update_circuit_db_param_values(circuit_db, resolver)

In [40]:
killer.remove_irrelevant_gates(cost, minimized_circuit_db)

ValueError: not enough values to unpack (expected 3, got 2)

In [39]:
killer.model_class.__name__

'QNN_VQE'

In [8]:
batch_circuits, trainable_symbols, trainable_params_value = prepare_optimization_discrimination(translator, circuit_db, etas)
cost, resolver, training_history = minimizer.minimize(batch_circuits, symbols = trainable_symbols, parameter_values = trainable_params_value )

KeyError: 'channel_param'

In [7]:
killer.remove_irrelevant_gates(circuit_db)

TypeError: remove_irrelevant_gates() missing 1 required positional argument: 'circuit_db'

In [43]:
mcircuit, mcircuit_db = translator.give_circuit(mutated_db)

In [52]:
check_symbols_ordered(mcircuit_db)
def repeated_symbols(mcircuit):
    ll=list(mcircuit["symbol"].dropna())
    return len(ll) != len(set(ll))

In [53]:
repeated_symbols(mcircuit_db)

False

In [33]:
simplified_db, s = simplifier.reduce_circuit(mcircuit_db)

simplified using  <bound method Simplifier.rule_4 of <utilities.simplifier.Simplifier object at 0x7f83607e7240>>
simplified using  <bound method Simplifier.rule_4 of <utilities.simplifier.Simplifier object at 0x7f83607e7240>>
simplified using  <bound method Simplifier.rule_4 of <utilities.simplifier.Simplifier object at 0x7f83607e7240>>
simplified using  <bound method Simplifier.rule_4 of <utilities.simplifier.Simplifier object at 0x7f83607e7240>>
simplified using  <bound method Simplifier.rule_4 of <utilities.simplifier.Simplifier object at 0x7f83607e7240>>
simplified using  <bound method Simplifier.rule_4 of <utilities.simplifier.Simplifier object at 0x7f83607e7240>>
simplified using  <bound method Simplifier.rule_5 of <utilities.simplifier.Simplifier object at 0x7f83607e7240>>
simplified using  <bound method Simplifier.rule_5 of <utilities.simplifier.Simplifier object at 0x7f83607e7240>>
simplified using  <bound method Simplifier.rule_5 of <utilities.simplifier.Simplifier object at 